In [1]:
from init import *
import time

In [2]:
def BuildModel_Simple():
    m = Model()
    m.TimeStart = 0
    m.TimeSteps = 10
    m.NumPaths = 10000
    m.Add(IndependentGaussian())
    m.Add(BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
    return m

In [3]:
m = BuildModel_Simple()
m.evaluations.append(EvaluationPoint(0,2))
m.json()

'{"TimeStart": 0, "TimeSteps": 10, "NumPaths": 10000, "updaters": [{"name": "IndependentGaussian", "refs": []}, {"name": "BrownianMotion", "start": 0.1, "args": [0.2, 2]}], "evaluations": [{"state": 0, "time": 2, "histograms": []}], "RandomSeed": -1, "RunTimeoutSeconds": 1, "MemoryLimitKB": 64}'

In [4]:
r = requests.post(f'{server}/model',m.json())
r.json()

{'histograms': [],
 'mean': [0.4738466441631317],
 'names': ['BrownianMotion'],
 'npaths': [10000],
 'run_time_seconds': {'mean': 0.00029289635131135583,
  'n': 1024,
  'skewness': None,
  'stddev': 6.900069138282561e-07},
 'skewness': [-0.003976175561547279],
 'stddev': [2.808436632156372],
 'time_points': [2.0],
 'time_steps': [10]}

In [5]:
m = BuildModel_Simple()
m.NumPaths = 1000000
m.TimeSteps = 10000
m.RunTimeoutSeconds = 16.7
for t in [1,2,5,10]:
    m.evaluations.append(EvaluationPoint(0,t))
time_start = time.time()
r = requests.post(f'{server}/model',m.json())
time_end = time.time()
r.json()

{'error': ['../src/core/Model.cpp Model& Model::Run()',
  'Statistics model_version5(Cuda::StreamGuard&, KernelModelArgs&)',
  'Model execution timeout is reached.']}

In [6]:
print(f'{m.NumPaths*m.TimeSteps/(time_end-time_start)*1e-9:.1f} nanoseconds to make a single time step')

10.5 nanoseconds to make a single time step


# Two processes, Three evaluations

In [7]:
def TwoProcessesModel ():
    m = Model()
    m.TimeStart = 0
    m.TimeSteps = 10
    m.NumPaths = 10000
    m.Add(IndependentGaussian())
    m.Add(BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
    m.Add(BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
    return m

In [8]:
model = Model()
model.TimeStart = 0
model.TimeSteps = 10
model.NumPaths = 10000
model.Add(IndependentGaussian())
model.Add(BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
model.Add(BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
model.TimeSteps = 10000
for t in [1,2,5]:
    model.evaluations.append(EvaluationPoint(state=0,time=t))
r = requests.post(f'{server}/model',model.json())
r.json()

{'histograms': [],
 'mean': [0.30863597989082336,
  0.27094054222106934,
  0.516170084476471,
  0.44071078300476074,
  1.1702731847763062,
  1.0227972269058228],
 'names': ['BrownianMotion', 'BrownianMotion'],
 'npaths': [10000, 10000, 10000, 10000, 10000, 10000],
 'run_time_seconds': {'mean': 0.2214757353067398,
  'n': 1024,
  'skewness': None,
  'stddev': 1.172416432382306e-06},
 'skewness': [-0.003529146546497941,
  -0.031775668263435364,
  0.027231745421886444,
  -0.002400784520432353,
  -0.0014919124078005552,
  0.006334626115858555],
 'stddev': [1.9982267618179321,
  2.0086891651153564,
  2.8414852619171143,
  2.813828945159912,
  4.507228374481201,
  4.435503005981445],
 'time_points': [1.0, 2.0, 5.0],
 'time_steps': [2000, 4000, 10000]}

In [9]:
er = EvaluationResults(r.json(),model)
er

2 states with 3 evaluations

In [10]:
er.df()

,name,title,state,time,step,npaths,mean,mean_error,stddev,skewness
0,BrownianMotion,,0,1.0,2000,10000,0.308636,0.019982,1.998227,-0.003529
1,BrownianMotion,,1,1.0,2000,10000,0.270941,0.020087,2.008689,-0.031776
2,BrownianMotion,,0,2.0,4000,10000,0.516170,0.028415,2.841485,0.027232
3,BrownianMotion,,1,2.0,4000,10000,0.440711,0.028138,2.813829,-0.002401
4,BrownianMotion,,0,5.0,10000,10000,1.170273,0.045072,4.507228,-0.001492
5,BrownianMotion,,1,5.0,10000,10000,1.022797,0.044355,4.435503,0.006335


In [11]:
logging.info('The End')

2022-11-03 00:02:45,364 [INFO] The End
